In [1]:
import sys
paths = [
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/util",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/config",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common/ds",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_intrinio",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common/web",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_portfolio",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/test/selenium/api",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/test/selenium",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/estypes",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/esmappings",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/esindexes",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/utils",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch_ingestion/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch_ingestion",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/util",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_quandl",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_snp",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/public",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_barchart",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_redis",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/models",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_twitter",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_edgar",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_fred",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_oecd",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_websocket",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/util/serverless/elasticsearch/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/symbol_mapping",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/sbt_elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_cryptocurrency",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/user",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_cbonds",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_brokers"
]

In [2]:
sys.path += paths
# sys.path.append("C:\\Users\\tbui\\Documents\\Stansberry-Terminal-MkI\\services\\screener_sbanalysis_api\\")

In [3]:

from pandas.io import sql as psql

In [4]:
from lib.connected_db import PgsqlExecutor

In [5]:
research_db = {
        "host": "dev-df-postgresql.csodrrohkuas.us-east-1.rds.amazonaws.com",
        "port": 5432,
        "user": "df_user",
        "credentials": "Piq914sT",
        "database": "datafactory"
      }
snpsource = {
        "host": "snp-instance-1-us-east-1b.csodrrohkuas.us-east-1.rds.amazonaws.com",
        "port": 5432,
        "user": "snpx_read",
        "credentials": "Nsp563Xz",
        "database": "xpressfeed"
      }

In [6]:
db = PgsqlExecutor(snpsource)
model_db = PgsqlExecutor(research_db)

# Long investment - consistent growth up

In [32]:
def get_long_stock_data(year):
    sql = """

    SELECT (market_dcf - price) / price , id, symbol, exchangesymbol, tradingitemid, price, pricing_date, 
    average_growth_rate, adjusted_growth, is_yearly_consistent_rate, 
    dcf, market_dcf, adjusted_dcf, adjusted_dcf_to_price, year_min_price, 
    year_max_price, year_end_price, end_pricing_date, dcf_ai_score, 
    predict_direction, fiscal_year

    from model.dcf_values_rank
    where fiscal_year = %s
    and adjusted_dcf_to_price != 'nan'
    and (market_dcf - price) > 0
    order by dcf_ai_score desc
    limit 50

    """
    data = model_db.fetch_all_rows(sql, args=[year])
    return data

def get_short_stock_data(year):
    """
    You don't want to short pennies stock, you only care when stock > 10
    """
    sql = """

    SELECT (market_dcf - price) / price , id, symbol, exchangesymbol, tradingitemid, price, pricing_date, 
    average_growth_rate, adjusted_growth, is_yearly_consistent_rate, 
    dcf, market_dcf, adjusted_dcf, adjusted_dcf_to_price, year_min_price, 
    year_max_price, year_end_price, end_pricing_date, dcf_ai_score, 
    predict_direction, fiscal_year

    from model.dcf_values_rank
    where fiscal_year = %s
    and adjusted_dcf_to_price != 'nan'
    and (market_dcf - price) < 0
    
    
    and price > 10
    order by dcf_ai_score
    limit 50

    """
    data = model_db.fetch_all_rows(sql, args=[year])
    return data
    


In [33]:
REPORT = """
**********- Score: {score} - {symbol} - {exchange}: {start_price} on {start_date} - *********
End date: {end_date} - price: {end_price} - min: {min_price} - max: {max_price}
Profit: {perf_profit} - max potential: {perf_potential} 
Average growth rate: {avg_growth}, consistent: {is_consistent}, project growth: {adj_growth}
DCF: {dcf}, market_dcf: {market_dcf}, adjusted_dcf: {adj_dcf}, dcf_to_price : {dcf_to_price}

"""
import numpy

def report_all(year):
    win = lose = 0
    profits = []
    potentials = []
    data = get_long_stock_data(year)
    for row in data.query_data:
        profit = round(
            (row.year_end_price - row.price) * 100 / row.price
            ,1)
        potential = round(
            (row.year_max_price - row.price) * 100 / row.price
            ,1)
        if profit > 0:
            win += 1
        else:
            lose += 1
        profits.append(profit)
        potentials.append(potential)

        report_statement = REPORT.format(
            score=row.dcf_ai_score,
            symbol = row.symbol,
            exchange = row.exchangesymbol,
            start_price = row.price,
            start_date = row.pricing_date,
            end_date=row.end_pricing_date,
            end_price=row.year_end_price,
            min_price=row.year_min_price,
            max_price=row.year_max_price,
            perf_profit=profit,
            perf_potential=potential,
            avg_growth = row.average_growth_rate,
            is_consistent = row.is_yearly_consistent_rate,
            adj_growth = row.adjusted_growth,
            dcf=row.dcf,
            market_dcf=row.market_dcf,
            adj_dcf=row.adjusted_dcf,
            dcf_to_price =row.adjusted_dcf_to_price




        )

        print(report_statement)

    print("*** Average Profit: {}% , Potential: {}% *** -  win: {} %".format(
        numpy.mean(profits), numpy.mean(potentials), round(win/ (win+lose), 2)
    ))
    return numpy.mean(profits), numpy.mean(potentials), round(win/ (win+lose), 2)

def report_short_all(year):
    win = lose = 0
    profits = []
    potentials = []
    data = get_short_stock_data(year)
    for row in data.query_data:
        profit = round(
            (row.price - row.year_end_price) * 100 / row.price
            ,1)
        potential = round(
            (row.price - row.year_min_price) * 100 / row.price
            ,1)
        if profit > 0:
            win += 1
        else:
            lose += 1
        profits.append(profit)
        potentials.append(potential)

        report_statement = REPORT.format(
            score=row.dcf_ai_score,
            symbol = row.symbol,
            exchange = row.exchangesymbol,
            start_price = row.price,
            start_date = row.pricing_date,
            end_date=row.end_pricing_date,
            end_price=row.year_end_price,
            min_price=row.year_min_price,
            max_price=row.year_max_price,
            perf_profit=profit,
            perf_potential=potential,
            avg_growth = row.average_growth_rate,
            is_consistent = row.is_yearly_consistent_rate,
            adj_growth = row.adjusted_growth,
            dcf=row.dcf,
            market_dcf=row.market_dcf,
            adj_dcf=row.adjusted_dcf,
            dcf_to_price =row.adjusted_dcf_to_price




        )

        print(report_statement)

    print("*** Average Profit: {}% , Potential: {}% *** -  win: {} %".format(
        numpy.mean(profits), numpy.mean(potentials), round(win/ (win+lose), 2)
    ))
    return numpy.mean(profits), numpy.mean(potentials), round(win/ (win+lose), 2)


# Including Pennies Stock - 2019

In [34]:
report_all(2019)


**********- Score: 54.20000076293945 - NVIV - NasdaqCM: 6.747006 on 2019-12-31 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 1.65 - min: 1.2248 - max: 10.05001
Profit: -75.5 - max potential: 49.0 
Average growth rate: 0.187, consistent: 0, project growth: 0.001
DCF: 1795.6, market_dcf: 1184.3, adjusted_dcf: 740.1, dcf_to_price : 10869.31



**********- Score: 52.79999923706055 - HDSN - NasdaqCM: 0.9775 on 2019-12-31 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 1.0 - min: 0.58 - max: 1.07
Profit: 2.3 - max potential: 9.5 
Average growth rate: 0.335, consistent: 0, project growth: 0.01
DCF: 4.6, market_dcf: 9.8, adjusted_dcf: 6.3, dcf_to_price : 544.5



**********- Score: 45.599998474121094 - JAKK - NasdaqGS: 1.03 on 2019-12-31 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 0.7825 - min: 0.311 - max: 1.14
Profit: -24.0 - max potential: 10.7 
Average growth rate: -0.101, consistent: 0, project growth: -0.167
DCF: 550.6, market_dcf: 11.6, adjuste

(-5.069999999999999, 44.23400000000001, 0.32)

# 2018

In [35]:
report_all(2018)


**********- Score: 61.20000076293945 - TNP - NYSE: 2.66 on 2018-12-31 00:00:00 - *********
End date: 2019-12-31 00:00:00 - price: 4.36 - min: 2.66 - max: 4.36
Profit: 63.9 - max potential: 63.9 
Average growth rate: 0.421, consistent: 0, project growth: 0.105
DCF: 4145.7, market_dcf: 1122.3, adjusted_dcf: 1174.1, dcf_to_price : 44039.1



**********- Score: 58.20000076293945 - RAVE - NasdaqCM: 0.8775 on 2018-12-31 00:00:00 - *********
End date: 2019-12-31 00:00:00 - price: 1.65 - min: 0.8775 - max: 3.11
Profit: 88.0 - max potential: 254.4 
Average growth rate: -0.013, consistent: 0, project growth: -0.023
DCF: 6.2, market_dcf: 9.4, adjusted_dcf: 6.5, dcf_to_price : 640.74



**********- Score: 54.5 - AVID - NasdaqGS: 4.75 on 2018-12-31 00:00:00 - *********
End date: 2019-12-31 00:00:00 - price: 8.58 - min: 4.41 - max: 10.32
Profit: 80.6 - max potential: 117.3 
Average growth rate: -0.715, consistent: 0, project growth: -1.075
DCF: 494.4, market_dcf: 10.0, adjusted_dcf: 1275.9, dcf_to_

(47.558, 95.04200000000003, 0.9)

In [36]:
initial_investment = 1000
potential_gain = initial_investment
yearly_cumulated_gain = initial_investment
potential_yearly_gains = []
profit_yearly_gains = []
win_lose = []

for year in range(2005, 2010):
    print (year)
    profit_per, potential_per, win = report_all(year)
    profit_yearly_gains.append(profit_per)
    potential_yearly_gains.append(potential_per)
    
    yearly_cumulated_gain = yearly_cumulated_gain * (1 + profit_per/100)
    potential_gain = potential_gain *(1 + potential_per/100)
    win_lose.append(win)



2005

**********- Score: 77.80000305175781 - EGAN - NasdaqCM: 0.72 on 2005-12-31 00:00:00 - *********
End date: 2006-12-29 00:00:00 - price: 1.31 - min: 0.72 - max: 1.85
Profit: 81.9 - max potential: 156.9 
Average growth rate: 0.966, consistent: 0, project growth: 0.449
DCF: 1241.4, market_dcf: 64.1, adjusted_dcf: 267.2, dcf_to_price : 37011.11



**********- Score: 60.20000076293945 - APWC - NasdaqGM: 1.5 on 2005-12-31 00:00:00 - *********
End date: 2006-12-29 00:00:00 - price: 2.75 - min: 0.8 - max: 3.2
Profit: 83.3 - max potential: 113.3 
Average growth rate: 0.458, consistent: 0, project growth: 0.028
DCF: 7.2, market_dcf: 16.0, adjusted_dcf: 11.6, dcf_to_price : 673.33



**********- Score: 43.0 - OII - NYSE: 12.445 on 2005-12-31 00:00:00 - *********
End date: 2006-12-29 00:00:00 - price: 19.85 - min: 12.795 - max: 23.575
Profit: 59.5 - max potential: 89.4 
Average growth rate: -0.036, consistent: 0, project growth: -0.06
DCF: 855.5, market_dcf: 15.5, adjusted_dcf: 958.6, dcf_to_


**********- Score: 30.0 - GALT - NasdaqCM: 2.700001 on 2006-12-31 00:00:00 - *********
End date: 2007-12-31 00:00:00 - price: 4.200001 - min: 1.5 - max: 8.340003
Profit: 55.6 - max potential: 208.9 
Average growth rate: -0.226, consistent: 0, project growth: -0.346
DCF: 27.5, market_dcf: 14.9, adjusted_dcf: 45.2, dcf_to_price : 1574.07



**********- Score: 27.299999237060547 - HEES - NasdaqGS: 24.77 on 2006-12-31 00:00:00 - *********
End date: 2007-12-31 00:00:00 - price: 18.88 - min: 15.12 - max: 30.47
Profit: -23.8 - max potential: 23.0 
Average growth rate: -0.243, consistent: 0, project growth: -0.427
DCF: 954.7, market_dcf: 39.0, adjusted_dcf: 1897.4, dcf_to_price : 7560.07



**********- Score: 27.100000381469727 - SPXC - NYSE: 61.16 on 2006-12-31 00:00:00 - *********
End date: 2007-12-31 00:00:00 - price: 102.85 - min: 60.98 - max: 109.88
Profit: 68.2 - max potential: 79.7 
Average growth rate: 0.048, consistent: 0, project growth: 0.0
DCF: 91.2, market_dcf: 95.8, adjusted_dcf


**********- Score: 68.0999984741211 - AAME - NasdaqGM: 0.742 on 2008-12-31 00:00:00 - *********
End date: 2009-12-31 00:00:00 - price: 1.28 - min: 0.45 - max: 1.69
Profit: 72.5 - max potential: 127.8 
Average growth rate: 2.278, consistent: 0, project growth: 1.38
DCF: 2347.0, market_dcf: 7.6, adjusted_dcf: 451.5, dcf_to_price : 60749.06



**********- Score: 59.5 - AAWW - NasdaqGS: 18.9 on 2008-12-31 00:00:00 - *********
End date: 2009-12-31 00:00:00 - price: 37.25 - min: 10.03 - max: 38.18
Profit: 97.1 - max potential: 102.0 
Average growth rate: -50.333, consistent: 0, project growth: -75.997
DCF: 24488344105.9, market_dcf: 152.9, adjusted_dcf: 185634297845.8, dcf_to_price : 982192051994.18



**********- Score: 53.70000076293945 - TGNA - NYSE: 8.0 on 2008-12-31 00:00:00 - *********
End date: 2009-12-31 00:00:00 - price: 14.85 - min: 1.95 - max: 15.63
Profit: 85.6 - max potential: 95.4 
Average growth rate: -0.007, consistent: 0, project growth: -0.014
DCF: -14.2, market_dcf: 69.8,

In [37]:
print("""
Initial Investment: ${}
Average yearly gain : {},
Average potential gain: {}
Cumulated potential capital: ${}
Cumulated capital yearly end: ${}
win = {} %
""".format(
    initial_investment,
    round(numpy.mean(profit_yearly_gains)),
    round(numpy.mean(potential_yearly_gains)),
    '{:0,.2f}'.format(yearly_cumulated_gain),
    '{:0,.2f}'.format(potential_gain), numpy.mean(win_lose)
    
))


Initial Investment: $1000
Average yearly gain : 43.0,
Average potential gain: 78.0
Cumulated potential capital: $4,329.69
Cumulated capital yearly end: $15,396.45
win = 0.712 %



# Shorting - current year 2019

In [38]:
report_short_all(2019)


**********- Score: -55.20000076293945 - STNG - NYSE: 39.34 on 2019-12-31 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 15.19 - min: 13.02 - max: 39.69
Profit: 61.4 - max potential: 66.9 
Average growth rate: 0.887, consistent: 0, project growth: 0.094
DCF: 137.0, market_dcf: 28.7, adjusted_dcf: 28.6, dcf_to_price : -27.3



**********- Score: -45.599998474121094 - DCTH - NasdaqCM: 21.25 on 2019-12-31 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 8.9 - min: 7.31 - max: 21.25
Profit: 58.1 - max potential: 65.6 
Average growth rate: -0.087, consistent: 0, project growth: -0.175
DCF: -33564.8, market_dcf: -34731.2, adjusted_dcf: -49036.5, dcf_to_price : -230860.0



**********- Score: -43.70000076293945 - UFCS - NasdaqGS: 43.73 on 2019-12-31 00:00:00 - *********
End date: 2020-06-18 00:00:00 - price: 27.51 - min: 23.2 - max: 46.73
Profit: 37.1 - max potential: 46.9 
Average growth rate: -0.828, consistent: 0, project growth: -1.393
DCF: -10169.1, market_dcf: -7

(14.632, 45.91000000000001, 0.72)

In [39]:
report_short_all(2018)


**********- Score: -65.0 - TOPS - NasdaqCM: 16.462 on 2018-12-31 00:00:00 - *********
End date: 2019-12-31 00:00:00 - price: 0.8 - min: 0.6525 - max: 22.0
Profit: 95.1 - max potential: 96.0 
Average growth rate: 0.047, consistent: 0, project growth: 0.006
DCF: -19358.3, market_dcf: -24148.9, adjusted_dcf: -15825.7, dcf_to_price : -96234.73



**********- Score: -54.70000076293945 - INPX - NasdaqCM: 143.550143 on 2018-12-31 00:00:00 - *********
End date: 2019-12-31 00:00:00 - price: 4.905004 - min: 1.710001 - max: 193.500193
Profit: 96.6 - max potential: 98.8 
Average growth rate: -0.63, consistent: 0, project growth: -0.954
DCF: -596581.0, market_dcf: -81276.8, adjusted_dcf: -1476169.7, dcf_to_price : -1028430.36



**********- Score: -41.79999923706055 - SBOW - NYSE: 23.64 on 2018-12-31 00:00:00 - *********
End date: 2019-12-31 00:00:00 - price: 9.9 - min: 7.43 - max: 25.54
Profit: 58.1 - max potential: 68.6 
Average growth rate: -0.192, consistent: 0, project growth: -0.378
DCF: -60

(-1.9960000000000004, 36.362, 0.52)

In [40]:
initial_investment = 1000
potential_gain = initial_investment
yearly_cumulated_gain = initial_investment
potential_yearly_gains = []
profit_yearly_gains = []
win_lose = []

for year in range(2005, 2010):
    print (year)
    profit_per, potential_per, win = report_short_all(year)
    profit_yearly_gains.append(profit_per)
    potential_yearly_gains.append(potential_per)
    
    yearly_cumulated_gain = yearly_cumulated_gain * (1 + profit_per/100)
    potential_gain = potential_gain *(1 + potential_per/100)
    win_lose.append(win)

print("""
Initial Investment: ${}
Average yearly gain : {},
Average potential gain: {}
Cumulated potential capital: ${}
Cumulated capital yearly end: ${}
win = {} %
""".format(
    initial_investment,
    round(numpy.mean(profit_yearly_gains)),
    round(numpy.mean(potential_yearly_gains)),
    '{:0,.2f}'.format(yearly_cumulated_gain),
    '{:0,.2f}'.format(potential_gain), numpy.mean(win_lose)
    
))

2005

**********- Score: -99.19999694824219 - WWR - NasdaqCM: 15839.987328 on 2005-12-31 00:00:00 - *********
End date: 2006-12-29 00:00:00 - price: 34800.01392 - min: 10980.004392 - max: 46079.963136
Profit: -119.7 - max potential: 30.7 
Average growth rate: -0.353, consistent: 0, project growth: -0.609
DCF: -1249924.5, market_dcf: -435495.6, adjusted_dcf: -2977339.0, dcf_to_price : -18896.35



**********- Score: -73.5999984741211 - CTIC - NasdaqCM: 26159.979072 on 2005-12-31 00:00:00 - *********
End date: 2006-12-29 00:00:00 - price: 20999.9832 - min: 13799.98896 - max: 27839.977728
Profit: 19.7 - max potential: 47.2 
Average growth rate: -0.217, consistent: 0, project growth: -0.423
DCF: -3414695.9, market_dcf: -2025416.1, adjusted_dcf: -7429161.2, dcf_to_price : -28498.96



**********- Score: -62.20000076293945 - AEMD - NasdaqCM: 247.500247 on 2005-12-31 00:00:00 - *********
End date: 2006-12-29 00:00:00 - price: 202.500202 - min: 150.00015 - max: 668.250668
Profit: 18.2 - max po


**********- Score: -53.599998474121094 - FCEL - NasdaqGM: 558.722457 on 2008-12-31 00:00:00 - *********
End date: 2009-12-31 00:00:00 - price: 541.442381 - min: 285.121254 - max: 695.523059
Profit: 3.1 - max potential: 49.0 
Average growth rate: 0.039, consistent: 0, project growth: 0.007
DCF: -14877.8, market_dcf: -18855.4, adjusted_dcf: -12886.3, dcf_to_price : -2406.39



**********- Score: -42.099998474121094 - SCON - NasdaqCM: 1817.820836 on 2008-12-31 00:00:00 - *********
End date: 2009-12-31 00:00:00 - price: 4392.00202 - min: 1530.000703 - max: 9180.004222
Profit: -141.6 - max potential: 15.8 
Average growth rate: -0.098, consistent: 0, project growth: -0.146
DCF: -1317.5, market_dcf: -1319.8, adjusted_dcf: -1257.5, dcf_to_price : -169.18



**********- Score: -37.0 - XOMA - NasdaqGM: 186.018601 on 2008-12-31 00:00:00 - *********
End date: 2009-12-31 00:00:00 - price: 209.660966 - min: 114.011401 - max: 354.035403
Profit: -12.7 - max potential: 38.7 
Average growth rate: -0.00

In [16]:
!ipython nbconvert --to html Reporting_Rank3_2020_06_16_adjust_dcf_to_market.ipynb

[TerminalIPythonApp] WARNING | Subcommand `ipython nbconvert` is deprecated and will be removed in future versions.
[TerminalIPythonApp] WARNING | You likely want to use `jupyter nbconvert` in the future
[NbConvertApp] Converting notebook Reporting_Rank3_2020_06_16_adjust_dcf_to_market.ipynb to html
[NbConvertApp] Writing 462717 bytes to Reporting_Rank3_2020_06_16_adjust_dcf_to_market.html
